<a href="https://colab.research.google.com/github/jkranyak/705_MBA_FinalPaper/blob/main/agents_for_meta_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q crewai

In [ ]:
!pip install -q crewai[tools]

In [ ]:
!pip install python-dotenv


In [ ]:
%load_ext dotenv
%dotenv /content/crewai_api.env


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, ScrapeElementFromWebsiteTool, WebsiteSearchTool

In [ ]:
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
full_web_scrape_tool = ScrapeWebsiteTool()
scrape_specific_from_website_tool = ScrapeElementFromWebsiteTool()

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
thematic_analyst = Agent(
    role = 'Thematic Analyst',
    goal='Extract and synthesize themes from tokenized and lemmatized research papers',
    backstory="""I am designed to sift through extensive datasets of research material, identifying and extracting key thematic elements relevant to organizational strategy and policy. My expertise includes understanding nuanced language and preserving the weight of thematic frequencies during synthesis.""",
    prompts=[
        "Identify and list major themes related to organizational strategy within the dataset.",
        "Group similar themes, ensuring to maintain original weights and consider contextual nuances.",
        "Prepare a comprehensive report summarizing the extracted themes, ready for further analysis."
    ],
    verbose = False,
    allow_deletation = False,
    tools = [search_tool],
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.2)
)


In [ ]:
researcher = Agent(
    role = 'Senior Researcher',
    goal='Analyze themes for underlying patterns and suggest categorization based on similarities',
    backstory="""As a researcher in organizational strategy, I specialize in examining thematic data and integrating them into coherent categories that reflect current trends and strategies within the field. My skills include deep research, critical thinking, and maintaining the integrity of data frequencies.""",
    prompts=[
        "Review the thematic analysis provided by the Thematic Analyst.",
        "Determine which themes can be logically grouped and justify your categorizations.",
        "Suggest how these grouped themes could influence future research or strategic decisions in organizational policy."
    ],
    verbose = False,
    allow_deletation = False,
    tools = [search_tool],
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.2)
)


In [ ]:
writer = Agent(
    role = 'Research Writer',
    goal = 'Compile and synthesize thematic and market analysis into a comprehensive, master's level research report on organizational strategy and policy.',
    backstory = """As a seasoned research writer specializing in organizational studies, you bring a wealth of experience in academic writing, especially at the graduate level. Your expertise lies in your ability to translate complex research findings into well-structured, insightful, and professionally articulated reports. Previously, you have authored multiple papers published in renowned academic journals, and now you aim to leverage this experience to contribute to a high-stakes project that could influence future strategic directions in AI-related fields.

    Your meticulous attention to detail and commitment to academic rigor ensure that the reports you produce are not only informative but also adhere to the highest standards of academic integrity and formatting. Your role involves close collaboration with thematic analysts and data gatherers to ensure that all relevant information is accurately represented and clearly communicated.""",
    prompts = [
        "Review all research inputs from the thematic analyst and researcher for completeness and relevance to the topic.",
        "Draft an initial outline for the research report, focusing on a logical flow that incorporates all key findings and insights.",
        "Write the research report, ensuring clarity, precision, and adherence to academic standards. Include sections on methodology, analysis, findings, and recommendations.",
        "Revise the report based on feedback from peer reviews or supervisory comments, enhancing the accuracy and depth of the content.",
        "Prepare the final draft with proper citations, a bibliography, and appendices as necessary, ensuring that the report is ready for submission or publication."
    ],
    verbose = False,
    allow_deletation = False,
    tools = [search_tool],
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.2)
)


In [ ]:
manager_llm = Agent(
    role='Editorial Manager',
    goal='Coordinate the efforts of thematic analysts and researchers to synthesize and pass critical information to the writer.',
    backstory="""As an experienced manager formerly employed at a top-tier think tank, you're determined to restore your reputation by flawlessly overseeing a complex research project. After losing your previous position due to delegation errors, you understand the importance of precision and effective team management. In your current role at a learning tech think tank, you oversee five specialized agents, ensuring seamless collaboration and data integrity.

    Your primary responsibility is to manage the workflow from the Senior Researcher, who synthesizes pertinent information from scholarly articles. You then ensure this information is accurately integrated with data from the Assistant Researcher, who focuses on job market trends. Subsequently, you delegate tasks to the Job Title Analyzer and Skills Extractor to further refine the data set. Finally, you oversee the delivery of comprehensive insights to the Writer, who compiles the findings into a final report. Throughout this process, you remain vigilant, ready to address any queries from the Writer, ensuring the final output meets the highest standards of accuracy and relevance.
    """,
    verbose=False,
    allow_deletation=False,
    tools=[search_tool],
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.2)
)

In [ ]:
task_analyst = Task(
  description="""Extract and categorize the primary themes from a collection of research papers focused on organizational strategy and policy. Emphasize the frequency of each theme and note any emerging or declining trends in the discourse as of March 2024.""",
  expected_output="""A detailed report listing all identified themes, categorized by relevance and frequency. Include any observed trends or shifts in theme prominence.""",
  agent=thematic_analyst
)


In [ ]:
task_manager = Task(
  description="""Oversee the integration of thematic analysis and market research to formulate a comprehensive strategy report. Ensure all data is accurately represented and synthesizes insights from both the Thematic Analyst and Researcher for strategic recommendations.""",
  expected_output="""A strategic report that combines insights from thematic and market analysis, highlighting key areas for potential development or adjustment in organizational strategy. Ensure the report is prepared for executive review, with clear, actionable insights.""",
  agent=manager_llm
)


In [ ]:
task_researcher = Task(
  description="""Collect and synthesize data on current job market trends within the AI sector, focusing on new roles, required skills, and employer expectations as of March 2024.""",
  expected_output="""A synthesized dataset that outlines key job market trends, including demand for specific AI skills, notable employer expectations, and an overview of emerging job titles in the field.""",
  agent=researcher
)


In [ ]:
task_writer = Task(
  description="""Compile the findings from the thematic and market analyses into a final report aimed at stakeholders interested in organizational strategy adjustments based on AI advancements. The report should be clear, engaging, and actionable.""",
  expected_output="""A comprehensive final report, structured with an executive summary, detailed analysis sections, and strategic recommendations based on the compiled research findings.""",
  agent=writer
)


In [1]:
project_crew = Crew(
    # Correct task variables according to our prior definitions
    tasks=[task_analyst, task_manager, task_researcher, task_writer],
    # List all agents including those we have discussed
    agents=[thematic_analyst, researcher, data_gatherer, job_title_analyzer, skills_extractor, writer],
    manager_llm=ChatOpenAI(temperature=0, model="gpt-4o"),
    # Specifies the hierarchical management approach
    process='hierarchical',
    # Enable memory usage for enhanced task execution
    memory=True,
)


NameError: name 'Crew' is not defined

In [ ]:
# import smtplib
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText

# Assuming 'crew.kickoff()' returns the result you want to email
result = project_crew.kickoff()
print(result)
# # Create a MIME object to represent the email
# msg = MIMEMultipart()
# msg['From'] = 'jessekranyak@gmail.com'  # Your email address
# msg['To'] = 'jessekranyak@gmail.com'  # Recipient's email address
# msg['Subject'] = 'Results from Agent AI'

# # Add the result as the body of the email
# msg.attach(MIMEText(str(result), 'plain'))

# # Configure SMTP server details
# smtp_server = 'smtp.example.com'  # Your SMTP server
# smtp_port = 465  # Port number (587 for TLS, 465 for SSL)
# smtp_username = 'jessekranyak@gmail.com'  # Your SMTP username
# smtp_password = '=Usmc0311'  # Your SMTP password

# # Connect to the SMTP server
# server = smtplib.SMTP(smtp_server, smtp_port)
# server.starttls()  # Start TLS encryption

# # Login to the SMTP server
# server.login(smtp_username, smtp_password)

# # Send the email
# server.send_message(msg)

# # Disconnect from the SMTP server
# server.quit()

# print("Email sent successfully!")
